### Notebook to demonstrate AutoML workflow for License Plate Recognition

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)

### Learning Objective

This AutoML notebook applies to identifying the optimal hyperparameters (e.g., learning rate, batch size, weight regularizer, number of layers, etc.) in order to obtain better accuracy results or converge faster on AI models for license plate recognition application.
- Take a pretrained model and choose automl algorithm/parameters to start AutoML train.
- At the end of an AutoML run, you will receive a config file that specifies the best performing model, along with the binary model file to deploy it to your application.


### AutoML Workflow

User starts with selecting model topology, create and upload dataset, configuring parameters, training with AutoML to comparing the model.

![image](https://raw.githubusercontent.com/vpraveen-nv/model_card_images/main/api/automl_workflow.png)


### Table of contents

1. [Create and upload datasets](#head-1)
1. [List the created datasets](#head-2)
1. [Dataset convert Action](#head-3)
1. [Create model](#head-4)
1. [List models](#head-5)
1. [Assign train, eval datasets](#head-6)
1. [Assign PTM](#head-7)
1. [Set AutoML related configurations](#head-8)
1. [Actions](#head-9)
1. [AutoML Train](#head-10)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import json
import os
import requests
import uuid
import time
from IPython.display import clear_output

### FIXME

1. Assign a workdir in FIXME1
2. Assign the ip_address and port_number in FIXME 2 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
3. Assign the ngc_api_key variable in FIXME 3
4. Choose between default and custom dataset in FIXME 4
6. Assign path of data_dir in FIXME 5
7. Choose between Bayesian and Hyperband automl_algorithm in FIXME 6

In [ ]:
# Define workspaces and other variables
# more information about lprnet can be found in https://docs.nvidia.com/tao/tao-toolkit/text/character_recognition/lprnet.html
workdir = "workdir_lprnet" # FIXME1
host_url = "http://<ip_address>:<port_number>" # FIXME2 example: https://10.137.149.22:32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key = "<ngc_api_key>" # FIXME3 example: zZYtczM5amdtdDcwNjk0cnA2bGU2bXQ3bnQ6NmQ4NjNhMDItMTdmZS00Y2QxLWI2ZjktNmE5M2YxZTc0OGyM
dataset_to_be_used = "default" # FIXME4 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset

In [ ]:
# Exchange NGC_API_KEY for JWT
response = requests.get(f"{host_url}/api/v1/login/{ngc_api_key}")
user_id = response.json()["user_id"]
print("User ID",user_id)
token = response.json()["token"]
print("JWT",token)

# Set base URL
base_url = f"{host_url}/api/v1/user/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}

In [ ]:
# Creating workdir
if not os.path.isdir(workdir):
    os.makedirs(workdir)

### Create datasets <a class="anchor" id="head-1"></a>

We will be using the `OpenALPR benchmark dataset` for the tutorial. The following script will download the dataset automatically and convert it to the format used by TAO.

**If using custom dataset; it should follow this dataset structure**
```
DATA_DIR
├── train
│   ├── characters.txt
│   ├── image
│   │   ├── image_name_1.jpg
│   │   ├── image_name_2.jpg
|   |   ├── ...
│   └── label
│       ├── image_name_1.txt
│       ├── image_name_2.txt
|       ├── ...
└── val
    ├── characters.txt
    ├── image
    │   ├── image_name_11.jpg
    │   ├── image_name_12.jpg
    |   ├── ...
    └── label
        ├── image_name_11.txt
        ├── image_name_12.txt
        ├── ...
```
The file name should be same for image and label folders

In [ ]:
data_dir = "lprnet_data" # FIXME5

In [ ]:
if dataset_to_be_used == "default":
    !python3 -m pip install --upgrade pip
    !python3 -m pip install "opencv-python>=3.4.0.12,<=4.5.5.64"    
    !mkdir -p $data_dir
    !bash ../dataset_prepare/lprnet/download_and_prepare_data.sh $data_dir

In [ ]:
country = "us" # us/ch; us for United States, ch for China
if dataset_to_be_used == "default":
    character_file_link = "https://api.ngc.nvidia.com/v2/models/nvidia/tao/lprnet/versions/trainable_v1.0/files/{}_lp_characters.txt".format(country)
    !wget -q -O $data_dir/train/characters.txt $character_file_link
    !cp $data_dir/train/characters.txt $data_dir/val/characters.txt

In [ ]:
!tar -C $data_dir/train/ -czf lprnet_train.tar.gz image label characters.txt
!tar -C $data_dir/val/ -czf lprnet_val.tar.gz image label characters.txt
!tar -C $data_dir/val/ -czf lprnet_test.tar.gz image

In [ ]:
train_dataset_path = "lprnet_train.tar.gz"
eval_dataset_path = "lprnet_val.tar.gz"
test_dataset_path = "lprnet_test.tar.gz"

In [ ]:
# Create train dataset
ds_type = "character_recognition"
ds_format = "lprnet"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())

dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Train dataset",
                       "description":"My train dataset with lprnet"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(train_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
# Create eval dataset
ds_type = "character_recognition"
ds_format = "lprnet"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())

eval_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Eval dataset",
                       "description":"My eval dataset with lprnet"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{eval_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(eval_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{eval_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

### List the created datasets <a class="anchor" id="head-2"></a>

In [ ]:
endpoint = f"{base_url}/dataset"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("id\t\t\t\t\t type\t\t\t format\t\t name")
for rsp in response.json():
    print(rsp["id"],"\t",rsp["type"],"\t",rsp["format"],"\t\t",rsp["name"])

### Create model <a class="anchor" id="head-4"></a>

In [ ]:
network_arch = "lprnet"
encode_key = "nvidia_tlt"
data = json.dumps({"network_arch":network_arch,"encryption_key":encode_key})

endpoint = f"{base_url}/model"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())

model_id = response.json()["id"]

### List models <a class="anchor" id="head-5"></a>

In [ ]:
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output

print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign train, eval datasets <a class="anchor" id="head-6"></a>

- Note: make sure the order for train_datasets is [source ID, target ID]
- eval_dataset is kept same as target for demo purposes
- inference_dataset is kept as target for chaining with hifigan finetune

In [ ]:
dataset_information = {"train_datasets":[dataset_id],
                       "eval_dataset":eval_dataset_id}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Assign PTM <a class="anchor" id="head-7"></a>

Search for pretrained models on NGC and assign it to the model

In [ ]:
# Get pretrained model for lprnet
model_list = f"{base_url}/model"
print(model_list)
response = requests.get(model_list, headers=headers)

response_json = response.json()

# Search for ptm with given ngc path
ptm_id = None
for rsp in response_json:
    if  rsp["network_arch"] == network_arch and rsp["additional_id_info"] == country:
        ptm_id = rsp["id"]
        print("Metadata for model with requested NGC Path")
        print(rsp)
        break
lprnet_ptm = ptm_id

In [ ]:
ptm_information = {"ptm":lprnet_ptm}
data = json.dumps(ptm_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### View hyperparameters that are enabled for AutoML by default

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/train/schema"

response = requests.get(endpoint, headers=headers)
specs = response.json()["automl_default_parameters"]

import json
print(json.dumps(specs, sort_keys=True, indent=4))

### Set AutoML related configurations <a class="anchor" id="head-8"></a>
Refer to the hyper-link to see the parameters supported by LPRNet and add more parameters if necessary in addition to the default automl enabled parameters: [LPRNet](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_action_specs.html#id21)

In [ ]:
# Choose automl algorithm between "Bayesian" and "HyperBand".
automl_algorithm="Bayesian" # FIXME6 example: Bayesian/HyperBand

metric="kpi"

additional_automl_parameters = [] #Refer to parameter list mentioned in the above links and add any extra parameter in addition to the default enabled ones
remove_default_automl_parameters = [] #Remove any hyperparameters that are enabled by default for AutoML

automl_information = {"automl_enabled":True,
                      "automl_algorithm":automl_algorithm,
                      "metric":metric,
                      "automl_add_hyperparameters":str(additional_automl_parameters),
                      "automl_remove_hyperparameters":str(remove_default_automl_parameters)
                     }
data = json.dumps(automl_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
import json
print(json.dumps(response.json(), sort_keys=True, indent=4))

### Actions <a class="anchor" id="head-9"></a>

For all actions:
1. Get default spec schema and derive the default values
2. Modify defaults if needed
3. Post spec dictionary to the service
4. Run model action
5. Monitor job using retrieve
6. Download results using job download endpoint (if needed)

In [ ]:
job_map = {}

### AutoML Train <a class="anchor" id="head-10"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/train/schema"

response = requests.get(endpoint, headers=headers)
print(response)

#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]

import json
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
specs["training_config"]["num_epochs"] = 24

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/train"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
import json
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = None
actions = ["train"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["train"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor automl job status by repeatedly running this cell
# Training times for different models benchmarked on 1 GPU V100 machine can be found here: https://docs.nvidia.com/tao/tao-toolkit/text/automl/automl.html#results-of-automl-experiments

job_id = job_map['train']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
## To Stop an AutoML JOB
#    1. Stop the 'Monitor automl job status by repeatedly running this cell' cell (the cell right before this cell) manually
#    2. Uncomment the snippet in the next cell and run the cell

In [ ]:
#job_id = job_map['train']
#endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

#response = requests.post(endpoint, headers=headers)

#print(response)
#print(response.json())

In [ ]:
## Resume AutoML

In [ ]:
# Uncomment the below snippet if you want to resume an already stopped AutoML job and then run the 'Monitor automl job status by repeatedly running this cell' cell above (4th cell above from this cell)
#job_id = job_map['train']
#endpoint = f"{base_url}/model/{model_id}/job/{job_id}/resume"

#response = requests.post(endpoint, headers=headers)

#print(response)
#print(response.json())

In [ ]:
# Download automl job contents once the above job shows "Done" status
# Download output of automl (lprnet) train (Note: will take time)
job_id = job_map["train"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    
print("Untarring")

# Untar to destination
tar_command = f'tar -xvf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# View best performing model's config, model file; Also view the results of all automl experiments
!python3 -m pip install pandas
import pandas as pd

best_model_path = f"{model_downloaded_path}/best_model"

if os.path.exists(best_model_path):        
    #List the binary model file
    print("\nCheckpoints for the best performing experiment")
    if os.path.exists(best_model_path+"/weights") and len(os.listdir(best_model_path+"/weights")) > 0:
        print(f"Folder: {best_model_path}/weights")
        print("Files:", os.listdir(best_model_path+"/weights"))
    else:
        print(f"Folder: {best_model_path}")
        print("Files:", os.listdir(best_model_path))

    experiment_artifacts = json.load(open(f"{best_model_path}/controller.json","r"))
    data_frame = pd.DataFrame(experiment_artifacts)
    # Print experiment id/number and the corresponding result
    print("\nResults of all experiments")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
        print(data_frame[["id","result"]])

    print("\nConfig/Spec file for the best performing experiment (recommendation_id.kitti with the maximum result value in the dataframe)")
    # List the recommendation config file of the best performing checkpoint(recommendation_id.kitti with the maximum result value in the dataframe)
    !ls {best_model_path}/*.kitti 